In [7]:
import numpy as np
import math 
import pandas as pd
from scipy import stats, optimize
import itertools
import matplotlib.pyplot as plt

# Management of dates and time
from datetime import datetime
from dateutil.relativedelta import relativedelta
from time import strftime, time
from timeit import timeit

# Curve Fitting
from nelson_siegel_svensson.calibrate import calibrate_nss_ols

# Pretty Table
from prettytable import PrettyTable

In [21]:
'''INSTANTANEOUS FORWARD FUNCTION DEFINITION
============================================='''
def instantaneousForwardRate(timeToMaturity, lambda_1, lambda_2, beta_0, beta_1, beta_2, beta_3):
    rate  = (beta_0 + beta_1 * np.exp(-timeToMaturity / lambda_1) + 
    beta_2 * timeToMaturity / lambda_1 * np.exp(-timeToMaturity / lambda_1) + 
    beta_3 * timeToMaturity / lambda_2 * np.exp(-timeToMaturity / lambda_2))
    return(rate)

def functionPhiG2PP(time, lambda_1, lambda_2, beta_0, beta_1, beta_2, beta_3, coeff_a, coeff_b, sigma, eta, rho):
    
    rate = instantaneousForwardRate(time, lambda_1, lambda_2, beta_0, beta_1, beta_2, beta_3) 
    num1 = np.power(sigma, 2) 
    den1 = 2 * np.power(coeff_a * (1 - np.exp(-coeff_a * time)), 2) 
            
    num2 = np.power(eta, 2) 
    den2 = 2 * np.power(coeff_b * (1 - np.exp(-coeff_b * time)), 2) 
    
    num3 = rho * eta * sigma 
    den3 =  (coeff_a * coeff_b) * (1 - np.exp(-coeff_a * time)) * (1 - np.exp(-coeff_b * time))
             
    result = num1/den1 + num2/den2 + num3/den3
    return(result)

def fCIR( time, kappa, theta, sigma, zVariable):
    #Renvoie le f CIR utilisé pour calculer le phi_t dans le modèle CIR2++
    
    h = np.sqrt(np.power(kappa, 2) + 2 *np.power(sigma, 2))
    num1 = 2 * kappa * theta * (np.exp(time * h) - 1)
    den1 = (2 * h + (kappa + h) * (np.exp(time * h) - 1))
    
    num2 = zVariable * (4 * np.power(h, 2) * np.exp(time * h))
    den2 = np.power((2 * h + (kappa + h) * (np.exp(time * h) - 1)), 2)
    
    result = num1/den1 + num2/den2
    return (result) 

def functionPhiCIR2PP(time, lambda_1, lambda_2, beta_0, beta_1, beta_2, beta_3, 
                      kappa_1, theta_1, sigma_1, xVariable, kappa_2, theta_2, sigma_2, yVariable):
    forward = instantaneousForwardRate(time, lambda_1, lambda_2, beta_0, beta_1, beta_2, beta_3)
    fCIR1 = fCIR(time, kappa_1, theta_1, sigma_1, xVariable)
    fCIR2 = fCIR(time, kappa_2, theta_2, sigma_2, yVariable)
    
    return(forward - fCIR1 - fCIR2)

In [22]:
'''FUNCTIONS A & B CIR2++
=========================='''
def AfunctionCIR2PP( initialDate, maturityDate, kappa, theta, sigma):
    #Fonction A dans le modèle CIR2++, utilisée pour simplifier les notations
    
    h = np.sqrt(np.power(kappa, 2) + 2 * np.power(sigma, 2))
    numerateurA = 2 * h * np.exp((kappa + h) * (maturityDate - initialDate) / 2)
    denominateurA = 2 * h + (kappa + h) * (np.exp((maturityDate - initialDate) * h) - 1)
    
    result = np.exp(((2 * kappa * theta) / np.power(sigma, 2))*np.log(numerateurA / denominateurA))
    return(result)

def BfunctionCIR2PP( initialDate, maturityDate, kappa, theta, sigma):
    #Fonction B dans le modèle CIR2++, utilisée pour simplifier les notations
    
    h = math.sqrt(np.power(kappa, 2) + 2 * np.power(sigma, 2))
    numerateurB = 2 * (math.exp((maturityDate - initialDate) * h) - 1)
    denominateurB = 2 * h + (kappa + h) * (math.exp((maturityDate - initialDate) * h) - 1)
    
    return (numerateurB / denominateurB)

def PfunctionCIR2PP( initialDate, maturityDate, kappa, theta, sigma, z):
    #Fonction P^CIR dans le modèle CIR2++, utilisée pour simplifier les notations
    A = AfunctionCIR2PP(initialDate, maturityDate, kappa, theta, sigma) 
    B = np.exp(-BfunctionCIR2PP(initialDate, maturityDate, kappa, theta, sigma) * z)
    return(A*B)

In [24]:
'''TAUX NSS + MARKET PRICES
============================'''
def tauxNSS( timeToMaturity, lambda_1, lambda_2, beta_0, beta_1, beta_2, beta_3):
    #Renvoie taux (composition continue) de maturité timeTomaturity avec les paramètres spécifiés, dans le modèle NSS
    if timeToMaturity == 0:
        return beta_0+beta_1
    else:
        return (beta_0 + 
    beta_1 * ((1 - np.exp(-timeToMaturity / lambda_1)) / (timeToMaturity / lambda_1)) +
    beta_2 * ((1 - np.exp(-timeToMaturity / lambda_1)) / (timeToMaturity / lambda_1) 
              - (np.exp(-timeToMaturity / lambda_1))) +
    beta_3 * ((1 - np.exp(-timeToMaturity / lambda_2)) / (timeToMaturity / lambda_2) 
              - (np.exp(-timeToMaturity / lambda_2))))

def marketInitialZeroCouponPrice( timeToMaturity, lambda_1, lambda_2, beta_0, beta_1, beta_2, beta_3):
    #Renvoie le prix du Zero Coupon (composition continue) de maturité timeTomaturity avec les paramètres spécifiés, dans le modèle NSS
    zeroCoupon = np.exp(-tauxNSS(timeToMaturity, lambda_1, lambda_2, 
                                 beta_0, beta_1, beta_2, beta_3) * timeToMaturity)
    return(zeroCoupon)

In [25]:
'''ZERO COUPON PRICE
============================'''
def zeroCouponPriceCIR2PP(initialDate, maturityDate, lambda_1, lambda_2, 
                          beta_0, beta_1, beta_2, beta_3, kappa_1, theta_1, 
                          sigma_1, kappa_2, theta_2, sigma_2, x_0, y_0, x_t, y_t):
    #Pricing des Zero-Coupon, modèle CIR2++
    
    phiXsi = ((marketInitialZeroCouponPrice(maturityDate, lambda_1, lambda_2, beta_0, beta_1, beta_2, beta_3)/ 
              marketInitialZeroCouponPrice(initialDate, lambda_1, lambda_2, beta_0, beta_1, beta_2, beta_3)) *
             (PfunctionCIR2PP(0, initialDate, kappa_1, theta_1, sigma_1, x_0) / 
              PfunctionCIR2PP(0, maturityDate, kappa_1, theta_1, sigma_1, x_0)) * 
             (PfunctionCIR2PP(0, initialDate, kappa_2, theta_2, sigma_2, y_0) / 
              PfunctionCIR2PP(0, maturityDate, kappa_2, theta_2, sigma_2, y_0)))
            
    pXsi = (PfunctionCIR2PP(initialDate, maturityDate, kappa_1, theta_1, sigma_1, x_t) * 
           PfunctionCIR2PP(initialDate, maturityDate, kappa_2, theta_2, sigma_2, y_t))
    
    return(phiXsi * pXsi)

def zeroCouponYieldCIR2PP( initialDate, maturityDate, lambda_1, lambda_2, 
                          beta_0, beta_1, beta_2, beta_3, kappa_1, theta_1, 
                          sigma_1, kappa_2, theta_2, sigma_2, x_0, y_0, x_t, y_t):
    #Fonction B dans le modèle CIR2++
    
    return -(np.log(zeroCouponPriceCIR2PP(initialDate, maturityDate, lambda_1, lambda_2, 
                                            beta_0, beta_1, beta_2, beta_3, kappa_1, theta_1,
                                            sigma_1, kappa_2, theta_2, sigma_2,
                                            x_0, y_0, x_t, y_t)) / (maturityDate - initialDate))


#Vectorize Functions
zeroCouponPriceCIR2PPVect = np.vectorize(zeroCouponPriceCIR2PP)
zeroCouponYieldCIR2PP = np.vectorize(zeroCouponYieldCIR2PP)

In [26]:
'''FORWARD SWAP RATE
====================='''
def forwardSwapRateCIR2PP(pricingDate, timeToMaturity, timeToTenor, deltaTime, lambda_1, lambda_2, 
                          beta_0, beta_1, beta_2, beta_3, kappa_1, theta_1, sigma_1, kappa_2,
                          theta_2, sigma_2, x_0, y_0, x_t, y_t):
    
    numerateur = zeroCouponPriceCIR2PP(pricingDate, pricingDate + timeToMaturity, lambda_1, lambda_2, 
                                       beta_0, beta_1, beta_2, beta_3, kappa_1, theta_1, sigma_1, 
                                       kappa_2, theta_2, sigma_2, x_0, y_0, x_t, y_t)
    
    numerateur = numerateur- zeroCouponPriceCIR2PP(pricingDate, pricingDate + timeToMaturity+timeToTenor, 
                                                  lambda_1, lambda_2, beta_0, beta_1, beta_2, beta_3, 
                                                  kappa_1, theta_1, sigma_1, kappa_2, theta_2,
                                                   sigma_2, x_0, y_0, x_t, y_t)
    
    paymentDates = pricingDate + timeToMaturity + np.range(timeToTenor//deltaTime) *deltaTime 
    
    PVBP = deltaTime*zeroCouponPriceCIR2PPVect(pricingDate,paymentDates, 
                                                        lambda_1, lambda_2, beta_0, beta_1, beta_2, beta_3, 
                                                        kappa_1, theta_1, sigma_1, kappa_2, theta_2, sigma_2,
                                                        x_0, y_0, x_t, y_t)
    
    denominateur = np.sum(PVBP)    
    return numerateur / denominateur

In [32]:
def vFunctionG2PP( initialDate, maturityDate, coeff_a, sigma, coeff_b, eta, rho):
    #Fonction V dans le modèle G2++, utilisée pour simplifier les notations
    
    return ((sigma / coeff_a) *(sigma / coeff_a)) * (maturityDate - initialDate + ((2 / coeff_a) * math.exp(-coeff_a * (maturityDate - initialDate))) - ((1 / (2 * coeff_a)) * math.exp(-2 * coeff_a * (maturityDate - initialDate))) - (3 / (2 * coeff_a))) + ((eta / coeff_b) *(eta / coeff_b)) * (maturityDate - initialDate + ((2 / coeff_b) * math.exp(-coeff_b * (maturityDate - initialDate))) - ((1 / (2 * coeff_b)) * math.exp(-2 * coeff_b * (maturityDate - initialDate))) - (3 / (2 * coeff_b))) + ((2 * rho * sigma * eta) / (coeff_a * coeff_b)) * (maturityDate - initialDate + ((math.exp(-coeff_a * (maturityDate - initialDate)) - 1) / (coeff_a)) + ((math.exp(-coeff_b * (maturityDate - initialDate)) - 1) / (coeff_b)) - ((math.exp(-(coeff_a + coeff_b) * (maturityDate - initialDate)) - 1) / (coeff_a + coeff_b)))

def AfunctionG2PP( initialDate, maturityDate, lambda_1, lambda_2, beta_0, beta_1, beta_2, beta_3, coeff_a, sigma, coeff_b, eta, rho):
    #Fonction A dans le modèle G2++, utilisée pour simplifier les notations
    
    return (marketInitialZeroCouponPrice(maturityDate, lambda_1, lambda_2, beta_0, beta_1, beta_2, beta_3) / marketInitialZeroCouponPrice(initialDate, lambda_1, lambda_2, beta_0, beta_1, beta_2, beta_3)) * (math.exp((1 / 2) * (vFunctionG2PP(initialDate, maturityDate, coeff_a, sigma, coeff_b, eta, rho) - vFunctionG2PP(0, maturityDate, coeff_a, sigma, coeff_b, eta, rho) + vFunctionG2PP(0, initialDate, coeff_a, sigma, coeff_b, eta, rho)))) 

def BfunctionG2PP( zReal, initialDate, maturityDate):
    #Fonction B dans le modèle G2++, utilisée pour simplifier les notations
    
    return (1 - exp(-zReal * (maturityDate - initialDate))) / zReal

def zeroCouponPriceG2PP( initialDate, maturityDate, lambda_1, lambda_2, beta_0, beta_1, beta_2, beta_3, coeff_a, sigma, coeff_b, eta, rho, xFactor, yFactor):
    #Princing des Zero-Coupon, modèle G2++
    
    return AfunctionG2PP(initialDate, maturityDate, lambda_1, lambda_2, beta_0, beta_1, beta_2, beta_3, coeff_a, sigma, coeff_b, eta, rho) * math.exp(-BfunctionG2PP(coeff_a, initialDate, maturityDate) * (xFactor)- BfunctionG2PP(coeff_b, initialDate, maturityDate) * (yFactor))

def zeroCouponYieldG2PP( initialDate, maturityDate, lambda_1, lambda_2, beta_0, beta_1, beta_2, beta_3, coeff_a, sigma, coeff_b, eta, rho, xFactor, yFactor):
    #Renvoie le taux Zero Coupon de maturité maturityDate à la date initialDate
    
    return -(math.log(zeroCouponPriceG2PP(initialDate, maturityDate, lambda_1, lambda_2, beta_0, beta_1, beta_2, beta_3, coeff_a, sigma, coeff_b, eta, rho, xFactor, yFactor)) / (maturityDate - initialDate))
